In [1]:
import pandas as pd
import numpy as np
import math
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [2]:
df = pd.read_csv('/Users/dhruv/code/export_yardi_jourentline.csv', lineterminator='\n')


In [3]:
df = df.drop(columns=[' GLNAME', ' PROPERTYNAME', ' BUILDING', ' SPECIALCIRCUMSTANCE', ' USAGEAMOUNT', ' USAGETYPE', ' UNIT', ' BALANCE', ' CONTROL'])

In [4]:
df.head()

,GLCODE,PROPERTY,DATE,PERIOD,DESCRIPTION,REFERENCE,AMOUNT,DEBITCREDIT,REMARKS
0,1110-0000,ap-mc11,09/09/2020,12/01/2020,Webster S8 (t0034597),165,8500.0,Credit,First buyout check
1,1110-0000,ap-mc11,09/29/2020,12/01/2020,Webster (t0034598),173,2500.0,Credit,
2,1110-0000,ap-mc11,10/27/2020,12/01/2020,Webster S8 (t0034597),183,9100.0,Credit,
3,1110-0000,ap-mc11,10/29/2020,12/01/2020,Jones (t0034602),187,3500.0,Credit,First Buyout Check
4,1110-0000,ap-mc11,11/02/2020,12/01/2020,Dow (t0034603),190,3500.0,Credit,First Buyout Check


In [5]:
df.columns = df.columns.map(lambda c: c.strip())

df['DATE'] = df['DATE'].map(lambda d: datetime.strptime(d, ' %m/%d/%Y'))
df['PERIOD'] = df['PERIOD'].map(lambda p: datetime.strptime(p, ' %m/%d/%Y'))
d


In [6]:
df.head()

,GLCODE,PROPERTY,DATE,PERIOD,DESCRIPTION,REFERENCE,AMOUNT,DEBITCREDIT,REMARKS
0,1110-0000,ap-mc11,2020-09-09,2020-12-01,Webster S8 (t0034597),165,8500,Credit,first buyout check
1,1110-0000,ap-mc11,2020-09-29,2020-12-01,Webster (t0034598),173,2500,Credit,
2,1110-0000,ap-mc11,2020-10-27,2020-12-01,Webster S8 (t0034597),183,9100,Credit,
3,1110-0000,ap-mc11,2020-10-29,2020-12-01,Jones (t0034602),187,3500,Credit,first buyout check
4,1110-0000,ap-mc11,2020-11-02,2020-12-01,Dow (t0034603),190,3500,Credit,first buyout check


In [7]:
catCols = [col for col in df.columns if df[col].dtype == 'object' or df[col].dtype == 'datetime64[ns]']


In [8]:
labeledDf = df.copy()

labelEncoder = LabelEncoder()

for col in catCols:
    labeledDf[col] = labelEncoder.fit_transform(df[col])
    
df = labeledDf.copy()


In [9]:
df.head()

,GLCODE,PROPERTY,DATE,PERIOD,DESCRIPTION,REFERENCE,AMOUNT,DEBITCREDIT,REMARKS
0,0,0,147,0,1507,2765,8500,0,2325
1,0,0,157,0,1506,2800,2500,0,0
2,0,0,174,0,1507,2835,9100,0,0
3,0,0,176,0,981,2842,3500,0,2325
4,0,0,180,0,747,2850,3500,0,2325


In [10]:
X = df.copy()
X = X.drop(['AMOUNT'], axis=1)

y = df['AMOUNT']

XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size=0.15, random_state=1) # THIS MESSES UP INDEXES

XTrain = XTrain.reset_index(drop=True)
yTrain = yTrain.reset_index(drop=True)
XTest = XTest.reset_index(drop=True)
yTest = yTest.reset_index(drop=True)


In [11]:
model = RandomForestRegressor(n_estimators=40)
model.fit(XTrain, yTrain)

#model = XGBRegressor(random_state=1, n_jobs=-1, learning_rate=0.92, n_estimators=10000)
#model.fit(XTrain, yTrain, early_stopping_rounds=4, eval_set=[(XTest, yTest)], verbose=False)

preds = pd.Series(model.predict(XTest))

In [12]:
print(model.score(XTrain, yTrain))
print(model.score(XTest, yTest))

0.9152553887685195
0.5879156723215635


In [13]:
comparison = pd.concat([preds, yTest], axis=1)
comparison.columns = ['predictions', 'values']
print(comparison)


       predictions  values
0       246.350000     177
1        81.325000     175
2       162.200714       4
3       760.368750     863
4      1599.825000     641
...            ...     ...
14530    46.716667      70
14531     1.925000       1
14532    38.350000      48
14533     4.025000       4
14534   327.225000     225

[14535 rows x 2 columns]
